In [1]:
#!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11/libexec/openjdk.jdk/Contents/Home"


THE CODE ABOVE IS TO INSTALL AND SET THE REQUIRED ENVIRONMENTS IN MACOS. To run this in windows, pyspark should be setup appropritely. 

In [2]:
import json
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("CommunityRDD").getOrCreate()
sc = spark.sparkContext

24/10/25 12:36:31 WARN Utils: Your hostname, Rheas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.116 instead (on interface en0)
24/10/25 12:36:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/25 12:36:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
json_file_path = '../Community Detection/10K.json'
with open(json_file_path, 'r') as f:
    community_data = json.load(f)

In [5]:
#Each community has nodes and edges, and we will assign a unique community ID to each
def add_community_id(community_list):
    for idx, community in enumerate(community_list, start=1):
        community['community_id'] = idx
    return community_list

In [6]:
community_data_with_ids = add_community_id(community_data)
community_rdd = sc.parallelize(community_data_with_ids)
type(community_rdd)

pyspark.rdd.RDD

In [7]:
#Create a key-value RDD where key = community_id, value = nodes and edges
community_kv_rdd = community_rdd.map(lambda community: (community['community_id'], {
    'nodes': community['nodes'], 
    'edges': community['edges']
}))

type(community_kv_rdd)

pyspark.rdd.PipelinedRDD

In [8]:
print(community_kv_rdd.take(5))

[(1, {'nodes': [371, 419, 204, 651, 134, 14, 447, 894, 72, 381, 173, 862, 812, 181, 621, 169, 628, 81, 281, 367, 773, 498, 486, 670, 774, 359, 108, 952, 513, 975, 313, 327, 73, 619, 178, 714, 615, 177, 458, 107, 345, 333, 162, 346, 742, 593, 18, 247, 518], 'edges': [{'node1': 371, 'node2': 419, 'begintijd': 20240211092539, 'eindtijd': 20240211105110}, {'node1': 371, 'node2': 812, 'begintijd': 20240412020025, 'eindtijd': 20240412053609}, {'node1': 371, 'node2': 670, 'begintijd': 20240606074446, 'eindtijd': 20240606110610}, {'node1': 419, 'node2': 518, 'begintijd': 20240302155007, 'eindtijd': 20240302220343}, {'node1': 204, 'node2': 651, 'begintijd': 20241206033837, 'eindtijd': 20241206042803}, {'node1': 134, 'node2': 486, 'begintijd': 20240527093355, 'eindtijd': 20240527100706}, {'node1': 134, 'node2': 345, 'begintijd': 20241112061756, 'eindtijd': 20241112122028}, {'node1': 14, 'node2': 419, 'begintijd': 20241124003819, 'eindtijd': 20241124054926}, {'node1': 14, 'node2': 621, 'begintijd

In [10]:
community_kv_rdd.saveAsTextFile("./communities_rdd.txt")